# Model 1D CNN followed by Global Average Pooling - All signals version

A stand-alone model intended to be imported as a .py file.

This version being used for PSG-Audio work.  Called by Evaluate_Model_PSG-Audio.ipynb.

Author:  [Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), [IMICS Lab](https://imics.wp.txstate.edu/), Texas State University, 2023

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODO:
* 

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras #added to save model
from tensorflow.keras import layers #format matches MNIST example
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
description = '1D_CNN_GAP_all_sigs.py Concat input, 1D CNNs then Global Avg Pool'
# Input params
evaluate = 'Respiratory'

# Model params
k1_num_filters = 25
k1_size = 51
k2_num_filters = 50
k2_size = 9
dropout = 0.5
mpool_size = 8

# Training params
BATCH_SIZE = 128
MAX_EPOCHS = 100
es_patience = 20 # early stopping patience
lrn_rate = 0.001 # learning rate, keras default is 0.001
lrn_verbose = 1 ##0 = silent, 1 = progress bar, 2 = one line per epoch

def build_model(x_train, y_train):
    """Builds 1D CNN followed by dense layer model.
    See .py for params.
    Returns: Model (Keras)"""
    print("build_model input shapes",x_train.shape, y_train.shape)
    # select labels and columns
    if evaluate == 'Respiratory':
        x_train = x_train[:,:,9:12] # flow, resp effort abd and tho
        #y_train = y_train[:,0:1] # respiratory labels only, keep shape though
    print("build_model modified shapes",x_train.shape, y_train.shape)
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model = keras.Sequential(
        [
            layers.Input(shape=x_train[0].shape),
            layers.Conv1D(filters=k1_num_filters, kernel_size=k1_size, activation='relu',input_shape=(x_train.shape[0],x_train.shape[1])),
            layers.Conv1D(filters=k2_num_filters, kernel_size=k2_size, activation='relu'),
            layers.GlobalAveragePooling1D(),
            layers.Dense(y_train.shape[1], activation='softmax')
            ])
    return model
def train_model(model,x_train, y_train, x_test, y_test):
    # see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    callback = EarlyStopping(monitor='val_loss', mode = 'min', patience=es_patience)
    opt = keras.optimizers.Adam(learning_rate=lrn_rate) # Keras default lr = 0.001
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    print("train_model input shapes",x_train.shape, y_train.shape,x_test.shape,y_test.shape)
    if evaluate == 'Respiratory':
        x_train = x_train[:,:,9:12] # flow, resp effort abd and tho
        #y_train = y_train[:,0:1] # respiratory labels only
        x_test = x_test[:,:,9:12] # flow, resp effort abd and tho
        #y_test = y_test[:,0:1] # respiratory labels only
    print("train_model modified shapes",x_train.shape, y_train.shape,x_test.shape,y_test.shape)
    history = model.fit(
        x_train,y_train,
        batch_size = BATCH_SIZE,
        epochs=MAX_EPOCHS,
        callbacks=[callback],
        validation_data=(x_test,y_test),
        verbose = lrn_verbose) #0 = silent, 1 = progress bar, 2 = one line per epoch
    return history, model